In [3]:
import api
import pandas as pd
eng = api.create_engine("sqlite:////Users/hj/Dropbox/real_estate/data/db/RealEstate.db")

In [21]:

with eng.connect() as conn:
    result = conn.execute(api.text("SELECT * FROM apt_raw WHERE 행정동코드 is NULL"))
    for row in result :
        print(row)

(8764281,)


In [17]:
import sqlite3
from sqlalchemy import create_engine, text

# SQLite 데이터베이스 연결 엔진 생성
engine = eng
# 데이터베이스 연결
with engine.connect() as conn:
    # 모든 테이블 이름 가져오기
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type ='table'"))
    tables = [row['name'] for row in result]

    # 각 테이블의 행 수 출력
    for table in tables:
        count_result = conn.execute(text(f"SELECT COUNT(*) AS row_count FROM {table}"))
        row_count = count_result.fetchone()['row_count']
        print(f"Table: {table}, Row count: {row_count}")

TypeError: tuple indices must be integers or slices, not str

In [13]:
create_table_query = """
CREATE TABLE test AS
WITH latest_codes AS (
    SELECT 
        법정동코드, 행정동코드, MAX(생성일자) AS 최신생성일자
    FROM 
        conn_code
    WHERE 
        말소일자 IS NULL OR 말소일자 > (
            SELECT 
                printf('%04d%02d%02d', 년, 월, 일) AS 거래일
            FROM apt_raw
        )
    GROUP BY 
        법정동코드
)
SELECT 
    a.*, 
    l.행정동코드
FROM 
    (SELECT 
         법정동시군구코드 || 법정동읍면동코드 AS 법정동코드, 
         printf('%04d%02d%02d', 년, 월, 일) AS 거래일,
         *
     FROM apt_raw) a
LEFT JOIN 
    latest_codes l
ON 
    a.법정동코드 = l.법정동코드
AND 
    a.거래일 >= l.최신생성일자
"""
# 데이터베이스 연결 및 테이블 생성 쿼리 실행
with eng.connect() as conn:
    conn.execute(api.text(create_table_query))


In [6]:
# 데이터베이스 연결 및 테이블 생성 쿼리 실행
with eng.connect() as conn:
    conn.execute(create_table_query)

NameError: name 'create_table_query' is not defined

In [24]:
# 행정동 법정동 코드 불러오기
import PublicDataReader as pdr

hdang_code = pdr.code_hdong() # 행정동코드
bdong_code = pdr.code_bdong() # 법정동코드
conn_code= pdr.code_hdong_bdong()

,시도코드,시도명,시군구코드,시군구명,행정동코드,읍면동명,법정동코드,동리명,생성일자,말소일자
0,11,서울특별시,11000,,1100000000,,1100000000,서울특별시,19880423,
1,11,서울특별시,11110,종로구,1111000000,,1111000000,종로구,19880423,
2,11,서울특별시,11110,종로구,1111000000,,1111090100,창신1동,19880423,19880423
3,11,서울특별시,11110,종로구,1111000000,,1111090200,창신2동,19880423,19880423
4,11,서울특별시,11110,종로구,1111000000,,1111090300,창신3동,19880423,19880423
...,...,...,...,...,...,...,...,...,...,...
55172,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042024,대리,20240118,
55173,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042025,거륜리,20240118,
55174,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042026,식도리,20240118,
55175,52,전북특별자치도,52800,부안군,5280042000,위도면,5280042027,상왕등리,20240118,
